### Importing the required libraries

In [4]:
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         822 KB

The following NEW packages will be INSTALLED:

    altair:  3.1.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


vincent-0.4.4        | 28 KB     | #################################

### Import the csv file containing Geo-ccordinates of prime places in Bangalore. Process the file to create a dataframe

In [6]:
import io
import requests
url="https://raw.githubusercontent.com/devsh-ds/Coursera_Capstone/master/Bangaloredata.csv"
BangaloreInfo=pd.read_csv(url)

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
0,Arabic College S.O,560045,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-25432636,NaN,Bangalore G.P.O.,77.6206,13.0291
1,Bellandur S.O,560103,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore South,Bangalore,KARNATAKA,080-28441625,NaN,H.A.L II Stage H.O,77.6760,12.9298
2,Domlur S.O,560071,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-25356966,NaN,H.A.L II Stage H.O,77.6359,12.9611
3,Dr. Shivarama Karanth Nagar S.O,560077,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,NaN,NaN,Bangalore G.P.O.,77.6293,13.0681
4,Fraser Town S.O,560005,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-25485125,NaN,Bangalore G.P.O.,77.6164,13.0005


In [7]:
DropList=["pincode","officeType","Deliverystatus","divisionname","regionname","circlename","Taluk","Districtname","statename","Telephone","Related Suboffice","Related Headoffice"]
BangaloreInfo.drop(DropList,axis = 1,inplace=True)

In [10]:
BangaloreInfo.head()

,officename,longitude,latitude
0,Arabic College S.O,77.6206,13.0291
1,Bellandur S.O,77.6760,12.9298
2,Domlur S.O,77.6359,12.9611
3,Dr. Shivarama Karanth Nagar S.O,77.6293,13.0681
4,Fraser Town S.O,77.6164,13.0005


In [16]:
for index, row in BangaloreInfo.iterrows():
    temp = row['officename']
    temp = temp.replace(" S.O"," ")
    BangaloreInfo.set_value(index,'officename',temp)
    #print(row['officename'][0:-4])
BangaloreInfo.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,officename,longitude,latitude
0,Arabic College,77.6206,13.0291
1,Bellandur,77.6760,12.9298
2,Domlur,77.6359,12.9611
3,Dr. Shivarama Karanth Nagar,77.6293,13.0681
4,Fraser Town,77.6164,13.0005


In [17]:
BangaloreInfo.rename(columns={'officename':'Area'},inplace=True)

In [18]:
BangaloreInfo.head()

,Area,longitude,latitude
0,Arabic College,77.6206,13.0291
1,Bellandur,77.6760,12.9298
2,Domlur,77.6359,12.9611
3,Dr. Shivarama Karanth Nagar,77.6293,13.0681
4,Fraser Town,77.6164,13.0005


In [20]:
print('The dataframe has {} Areas'.format(
        len(BangaloreInfo['Area'].unique())
    )
)

The dataframe has 56 Areas


### Creating a map of Bangalore and ploting all the areas

In [24]:
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area in zip(BangaloreInfo['latitude'], BangaloreInfo['longitude'], BangaloreInfo['Area']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


### Setup the credentials for Foursquare API access

In [35]:
CLIENT_ID = 'UMAT3RX5PETGERIAYGWHO5G1XUQO1KNV2PAF0YJCAYXP5S2B' # your Foursquare ID
CLIENT_SECRET = 'AX04JHBTZA5K4ZR0YPOK1DTY2TDLTCZ4YL540NBMGD2C2YBV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


def GetAllVenues(AreaName, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(AreaName, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

AllVenuesInfo = GetAllVenues(BangaloreInfo['Area'],BangaloreInfo['latitude'], BangaloreInfo['longitude'])

Your credentails:
CLIENT_ID: UMAT3RX5PETGERIAYGWHO5G1XUQO1KNV2PAF0YJCAYXP5S2B
CLIENT_SECRET:AX04JHBTZA5K4ZR0YPOK1DTY2TDLTCZ4YL540NBMGD2C2YBV
Arabic College 
Bellandur 
Domlur 
Dr. Shivarama Karanth Nagar 
Fraser Town 
G.K.V.K. 
H.A.L II Stage H.O
HighCourt 
Jeevanbhimanagar 
Kothanur 
Mahatma Gandhi Road 
Marathahalli Colony 
NAL 
New Thippasandra 
Sadashivanagar 
Sahakaranagar P.O 
Vimanapura 
Yelahanka 
Yelahanka Satellite Town 
Bangalore G.P.O. 
Ashoknagar  (Bangalore)
B Sk II Stage 
Bannerghatta Road 
Basavanagudi H.O
Bommanahalli  (Bangalore)
Bommasandra Industrial Estate 
Carmelram 
Chandapura 
Dharmaram College 
Electronics City 
Gottigere 
HSR Layout 
Hulimavu 
J P Nagar 
Jayanagar H.O
Jayangar III Block 
JP Nagar III Phase 
Koramangala VI Bk 
Mico Layout 
Padmanabhnagar 
St. John's Medical College 
Gayathrinagar 
Jalahalli East 
Jalahalli H.O
Mahalakshmipuram Layout 
Malleswaram 
Malleswaram West 
Mathikere 
Msrit 
Palace Guttahalli 
Rajajinagar H.O
Rajajinagar IVth Block 
Swi

In [36]:
for item in AllVenuesInfo['Venue Category'].unique():
    print(item)

Indian Restaurant
Furniture / Home Store
Fast Food Restaurant
Soccer Field
Café
Pizza Place
Shopping Mall
Kerala Restaurant
Gym
Multiplex
Athletics & Sports
Burger Joint
Food Court
Breakfast Spot
Juice Bar
Park
Andhra Restaurant
Grocery Store
Asian Restaurant
Dumpling Restaurant
Pakistani Restaurant
Middle Eastern Restaurant
Tea Room
Hyderabadi Restaurant
Ice Cream Shop
Bakery
Mediterranean Restaurant
Sandwich Place
BBQ Joint
Chinese Restaurant
Coffee Shop
South Indian Restaurant
Department Store
Flea Market
Vegetarian / Vegan Restaurant
Garden
Basketball Court
Farmers Market
Spa
Chocolate Shop
Pub
Nightclub
Deli / Bodega
Restaurant
Lounge
Cupcake Shop
Cocktail Bar
Multicuisine Indian Restaurant
Vietnamese Restaurant
Italian Restaurant
Brewery
Boutique
Music Venue
Bar
Liquor Store
Sports Bar
Accessories Store
Hotel
Tex-Mex Restaurant
Diner
Clothing Store
Speakeasy
Mexican Restaurant
College Bookstore
Convenience Store
Capitol Building
Tennis Stadium
Dog Run
Dessert Shop
Smoke Shop
Arts

### Selecting the venues that are related to sports, kids or office areas

In [37]:
VenuesOfInterest=['Soccer Field','Shopping Mall','Gym','Multiplex','Athletics & Sports','Food Court','Juice Bar','Park','Garden','Basketball Court','Spa','Tennis Stadium','Dog Run','Dessert Shop','Playground','Office','Sporting Goods Shop','Gym / Fitness Center','Stadium','Dance Studio']
print(len(VenuesOfInterest))

20


In [38]:
RelevantAreaInfo = pd.DataFrame()
for index, row in AllVenuesInfo.iterrows():
    if row['Venue Category'] in VenuesOfInterest:
        RelevantAreaInfo=RelevantAreaInfo.append(row)
        

In [39]:
RelevantAreaInfo.head()

,Area,Area Latitude,Area Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
3,Bellandur,12.9298,77.676,Kicks On Grass,Soccer Field,12.930045,77.679679
6,Bellandur,12.9298,77.676,Bangalore Central,Shopping Mall,12.926006,77.675301
8,Bellandur,12.9298,77.676,Sobha Lakeview Clubhouse,Gym,12.929097,77.672374
9,Bellandur,12.9298,77.676,PVR,Multiplex,12.926328,77.675347
12,Bellandur,12.9298,77.676,Playmania,Athletics & Sports,12.930281,77.673720


## Identifying the Top 3 areas where most number of relevant venues exist

In [40]:
RelevantAreaInfo.groupby('Area').count()

,Area Latitude,Area Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
Area,,,,,,
Ashoknagar (Bangalore),3,3,3,3,3,3
Bangalore G.P.O.,1,1,1,1,1,1
Bannerghatta Road,2,2,2,2,2,2
Basavanagudi H.O,1,1,1,1,1,1
Bellandur,7,7,7,7,7,7
Bommasandra Industrial Estate,1,1,1,1,1,1
Carmelram,1,1,1,1,1,1
Dharmaram College,1,1,1,1,1,1
Domlur,1,1,1,1,1,1


In [41]:
RelevantAreaInfo['Area'].value_counts()

Koramangala VI Bk                 8
Jayangar III Block                7
Bellandur                         7
H.A.L II Stage H.O                4
Mico Layout                       3
Mahatma Gandhi Road               3
Ashoknagar  (Bangalore)           3
HSR Layout                        3
Jalahalli H.O                     3
HighCourt                         3
Fraser Town                       3
Sadashivanagar                    2
NAL                               2
Jeevanbhimanagar                  2
G.K.V.K.                          2
JP Nagar III Phase                2
Bannerghatta Road                 2
Vyalikaval Extn                   2
Electronics City                  1
Vimanapura                        1
Yelahanka                         1
Marathahalli Colony               1
Domlur                            1
New Thippasandra                  1
Palace Guttahalli                 1
Basavanagudi H.O                  1
J P Nagar                         1
Mathikere                   

In [60]:
TopThreeAreas = ['Koramangala VI Bk ','Jayangar III Block ','Bellandur ']
FinalMap = pd.DataFrame()
for index, row in RelevantAreaInfo.iterrows():
    #print (type(row['Area']))
    if row['Area'] in TopThreeAreas:
        #print("yes")
        FinalMap=FinalMap.append(row)
        
#RelevantAreaInfo[RelevantAreaInfo.Area == "Koramangala VI Bk "]
        
#FinalMap.head()
#RelevantAreaInfo.head()


### Plotting the top three areas and corresponding venues to find right place for Yogurt Shop

In [62]:
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area in zip(FinalMap['Venue Latitude'], FinalMap['Venue Longitude'], FinalMap['Venue']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.
